In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print(f'Found GPU at: {device_name}')

# Alternative way to check
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


2025-05-18 10:21:40.597934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747563700.770092      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747563700.821099      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


I0000 00:00:1747563713.543023      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.6 MB/s eta 0:00:0000:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
!pip install transformers

# **PART 2**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [ ]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset
from torch.optim import AdamW
import pandas as pd

# Suppress transformer warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
from datasets import load_dataset

# Load the Tamil NER dataset
dataset = load_dataset("ai4bharat/indic_glue", "wiki-ner.ta")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

README.md:   0%|          | 0.00/49.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/292k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20466 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2586 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2611 [00:00<?, ? examples/s]

In [ ]:
# Preprocessing the dataset to align NER tags with tokens
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)  # Ignore these tokens in the loss
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])  # Only label first token of the word
        else:
            # Continue with I- tags for inner tokens, otherwise ignore
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels


def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]

    # Align labels with word ids for each example in the batch
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]

    # Add aligned labels to tokenized inputs
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs


train_dataset = train_data.map(preprocess_data, batched=True)
validation_dataset = validation_data.map(preprocess_data, batched=True)
test_dataset = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/20466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2586 [00:00<?, ? examples/s]

Map:   0%|          | 0/2611 [00:00<?, ? examples/s]

In [ ]:
# Load model
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model with more capacity for multilingual tasks
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=dataset['train'].features['ner_tags'].feature.num_classes)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c9187dd46bd7f6dfbcd3f96dd6d319bfc9fd03db4b19cf1a0e79591cd3cbf4e0
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Define compute_metrics function
def compute_metrics(pred):
    label_ids = pred.label_ids
    predictions = pred.predictions.argmax(-1)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[label_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                   for i in range(len(label_ids))]
    true_predictions = [[label_list[pred_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                        for i in range(len(predictions))]

    # Calculate metrics using seqeval
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
# A-LLRD Optimizer (fixed for mBERT and checkpointing)
def get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95):
    def compute_gradient_norms(model):
        norms = []
        for i, layer in enumerate(model.bert.encoder.layer):
            norm = 0
            for param in layer.parameters():
                if param.grad is not None:
                    norm += param.grad.norm().item() ** 2
            norms.append(np.sqrt(norm) if norm > 0 else 1e-8)
        return norms

    class ALLRDOptimizer(AdamW):
        def __init__(self, params, model, base_lr, min_decay, max_decay, betas=(0.9, 0.999), eps=1e-8, amsgrad=False, weight_decay=0.01, fused=False, capturable=False, differentiable=False, maximize=False, foreach=None):
            defaults = dict(
                lr=base_lr,
                betas=betas,
                eps=eps,
                amsgrad=amsgrad,
                weight_decay=weight_decay,
                fused=fused,
                capturable=capturable,
                differentiable=differentiable,
                maximize=maximize,
                foreach=foreach
            )
            super().__init__(params, **defaults)
            self.model = model
            self.base_lr = base_lr
            self.min_decay = min_decay
            self.max_decay = max_decay
            self.step_count = 0

        def step(self, closure=None):
            self.step_count += 1
            norms = compute_gradient_norms(self.model)
            norms = [min(max(norm, 1e-8), 5.0) for norm in norms]
            norms = torch.tensor(norms, device=next(self.model.parameters()).device)
            decay_factors = torch.softmax(-norms, dim=0).cpu().numpy()
            decay_factors = self.min_decay + (self.max_decay - self.min_decay) * decay_factors / decay_factors.max()
            if self.step_count % 100 == 0:
                print(f"Step {self.step_count}: Decay factors: {decay_factors.tolist()}")
            # Update learning rates of existing param_groups instead of rebuilding
            for i, param_group in enumerate(self.param_groups[:-1]):  # Exclude classifier group
                param_group['lr'] = self.base_lr * decay_factors[i]
            self.param_groups[-1]['lr'] = self.base_lr  # Classifier LR remains fixed
            super().step(closure)

    # Initialize parameter groups (only once)
    optimizer_grouped_parameters = [
        {
            "params": layer.parameters(),
            "lr": base_lr,
            "betas": (0.9, 0.999),
            "eps": 1e-8,
            "amsgrad": False,
            "weight_decay": 0.01,
            "fused": False,
            "capturable": False,
            "differentiable": False,
            "maximize": False,
            "foreach": None
        } for layer in model.bert.encoder.layer
    ] + [{
        "params": model.classifier.parameters(),
        "lr": base_lr,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
        "amsgrad": False,
        "weight_decay": 0.01,
        "fused": False,
        "capturable": False,
        "differentiable": False,
        "maximize": False,
        "foreach": None
    }]
    return ALLRDOptimizer(optimizer_grouped_parameters, model, base_lr, min_decay, max_decay)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to eval_strategy as requested
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,
    report_to="none",
    logging_steps=100,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    save_total_limit=1
)

# Initialize trainer (without early stopping)
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(get_optimizer(model, base_lr=2e-5, min_decay=0.85, max_decay=0.95), None)
)

# Train the model
trainer.train()
metrics = trainer.evaluate(test_dataset)
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

/tmp/ipykernel_35/772242240.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.157200,0.181474,0.950203,0.830722,0.862873,0.846492
2,0.088400,0.196429,0.951443,0.843869,0.851980,0.847905
3,0.106600,0.188238,0.958506,0.872776,0.880046,0.876396
4,0.064300,0.229642,0.956225,0.856267,0.877227,0.866620
5,0.047200,0.247187,0.956447,0.857878,0.875689,0.866692
6,0.045300,0.232269,0.956956,0.859917,0.879534,0.869615
7,0.038500,0.248006,0.956004,0.863443,0.883250,0.873234
8,0.025100,0.262030,0.959281,0.864841,0.886454,0.875514
9,0.030100,0.270106,0.957465,0.850410,0.890299,0.869897
10,0.023600,0.273710,0.958152,0.870988,0.879918,0.875430


Step 100: Decay factors: [0.9366770515817998, 0.9329505485144381, 0.9291238906595721, 0.9267396672796605, 0.9235314222508797, 0.9292175965720205, 0.9358973551292629, 0.9363760516996794, 0.9407496415653254, 0.9460694585518853, 0.95, 0.949480028787122]
Step 200: Decay factors: [0.9288283490158771, 0.9243365358386945, 0.9237796080205554, 0.9235141516216464, 0.9237198011068773, 0.9307266131295198, 0.9381133400759476, 0.9419092286977941, 0.9448435580626889, 0.9473135202443117, 0.95, 0.9498965185890159]
Step 300: Decay factors: [0.9403063607318972, 0.9382540618680131, 0.9355277884901074, 0.9324258832148458, 0.9286774138758062, 0.926567544158252, 0.9306778739655859, 0.9334993266810762, 0.9393971128455268, 0.9451858139748284, 0.95, 0.9497178021022361]
Step 400: Decay factors: [0.9348869267969415, 0.9334509721195666, 0.9317627904537253, 0.9296643710285616, 0.9291393017931899, 0.9313520069101967, 0.937201895871387, 0.9373639084167704, 0.9430266052302739, 0.9458777528521514, 0.95, 0.9496267879080

   eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1  \
0   0.342014       0.944504         0.85745     0.860077  0.858762   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0        11.361                  229.822                 28.783   10.0  


In [ ]:
# Inspect the log history to understand the structure of logs
for log in trainer.state.log_history:
    print(log)


{'loss': 0.6328, 'grad_norm': 6.452209949493408, 'learning_rate': 1.992262602579133e-05, 'epoch': 0.039077764751856196, 'step': 100}
{'loss': 0.3226, 'grad_norm': 16.198402404785156, 'learning_rate': 1.9844470496287613e-05, 'epoch': 0.07815552950371239, 'step': 200}
{'loss': 0.2796, 'grad_norm': 8.488667488098145, 'learning_rate': 1.9766314966783903e-05, 'epoch': 0.11723329425556858, 'step': 300}
{'loss': 0.2709, 'grad_norm': 6.1115264892578125, 'learning_rate': 1.968815943728019e-05, 'epoch': 0.15631105900742479, 'step': 400}
{'loss': 0.2318, 'grad_norm': 5.733156681060791, 'learning_rate': 1.9610003907776477e-05, 'epoch': 0.19538882375928096, 'step': 500}
{'loss': 0.2307, 'grad_norm': 6.60952615737915, 'learning_rate': 1.9531848378272764e-05, 'epoch': 0.23446658851113716, 'step': 600}
{'loss': 0.2116, 'grad_norm': 6.086686134338379, 'learning_rate': 1.945369284876905e-05, 'epoch': 0.27354435326299337, 'step': 700}
{'loss': 0.1873, 'grad_norm': 11.038026809692383, 'learning_rate': 1.9

In [ ]:
# Log validation metrics per epoch
print("\nValidation Metrics per Epoch:")
validation_metrics = []
for log in trainer.state.log_history:
    if "eval_f1" in log:
        epoch_metrics = {
            "epoch": log.get("epoch", None),
            "eval_loss": log.get("eval_loss", None),
            "eval_accuracy": log.get("eval_accuracy", None),
            "eval_precision": log.get("eval_precision", None),
            "eval_recall": log.get("eval_recall", None),
            "eval_f1": log.get("eval_f1", None)
        }
        validation_metrics.append(epoch_metrics)
validation_df = pd.DataFrame(validation_metrics)
print(validation_df)

# Evaluate on test dataset (overall metrics)
print("\nTest Metrics:")
test_metrics = trainer.evaluate(test_dataset)
test_metrics_summary = {
    "test_accuracy": test_metrics["eval_accuracy"],
    "test_precision": test_metrics["eval_precision"],
    "test_recall": test_metrics["eval_recall"],
    "test_f1": test_metrics["eval_f1"]
}
test_metrics_df = pd.DataFrame([test_metrics_summary])
print(test_metrics_df)

# Compute per-class metrics manually
print("\nPer-Class Test Metrics:")
model.eval()
all_true_labels = []
all_pred_labels = []

for batch in trainer.get_test_dataloader(test_dataset):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    labels = batch["labels"].cpu().numpy()
    for i in range(len(labels)):
        true_labels = [label_list[l] for l, p in zip(labels[i], predictions[i]) if l != -100]
        pred_labels = [label_list[p] for l, p in zip(labels[i], predictions[i]) if l != -100]
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)

class_report = classification_report([all_true_labels], [all_pred_labels], output_dict=True)
per_class_data = []
for label, metrics in class_report.items():
    if label in label_list:
        per_class_data.append({
            "label": label,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1-score": metrics["f1-score"],
            "support": metrics["support"]
        })
per_class_df = pd.DataFrame(per_class_data)
print(per_class_df)


Validation Metrics per Epoch:
    epoch  eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1
0     1.0   0.250192       0.925404        0.734773     0.781148  0.757251
1     2.0   0.210188       0.940847        0.790123     0.839344  0.813990
2     3.0   0.200103       0.943737        0.792581     0.858197  0.824085
3     4.0   0.235549       0.945182        0.817684     0.856557  0.836669
4     5.0   0.236798       0.950601        0.824439     0.873770  0.848388
5     6.0   0.289781       0.948162        0.844125     0.865574  0.854715
6     7.0   0.292874       0.950059        0.831505     0.869672  0.850160
7     8.0   0.315085       0.950510        0.814928     0.877049  0.844848
8     9.0   0.305811       0.952316        0.835285     0.877049  0.855658
9    10.0   0.325303       0.952046        0.824031     0.871311  0.847012
10   10.0   0.307089       0.952226        0.833977     0.870266  0.851735
11   10.0   0.307089       0.952226        0.833977     0.870266  0.8

In [ ]:
# Test model on example sentences
def test_model(sentences):
    model.eval()
    for sentence in sentences:
        tokenized_input = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        input_ids = tokenized_input["input_ids"].to(model.device)
        attention_mask = tokenized_input["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        predicted_labels = [label_list[p] for p in predictions[0]]
        print(f"\nTesting sentence: {sentence}")
        for token, label in zip(tokens, predicted_labels):
            if token not in tokenizer.special_tokens_map.values() and token.strip():
                print(f"Word: '{token}', Predicted Label: {label}")

test_sentences = [
    "என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.",
    "இது ஒரு புதிய தொழில்நுட்ப மேம்பாடு.",
    "தமிழ் மொழி தமிழ்நாட்டின் நிலையான மொழியாகும்.",
    "இத்தாவரம் சீனா, இந்தியா, ஜப்பான், பப்பாசியா, மைக்குரோனீசியா போன்ற நாடுகளில் காணப்படுகிறது。",
    "சைஃப் அலி கான், தீபிகா படுகோண், ஜான் ஆபிரகாம் (நடிகர்), ஜாக்குலின் பெர்னாண்டஸ், அனில் கபூர், அமீஷா பட்டேல், மற்றும் ரஜினிகாந்த் ஆகியோர்。"
]
test_model(test_sentences)


Testing sentence: என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.
Word: 'என்', Predicted Label: O
Word: 'பெயர்', Predicted Label: O
Word: 'வ', Predicted Label: B-PER
Word: '##ின்', Predicted Label: O
Word: '##ஸ்', Predicted Label: I-PER
Word: '##டன்', Predicted Label: I-PER
Word: 'ச', Predicted Label: O
Word: '##ர்', Predicted Label: I-PER
Word: '##ச்சி', Predicted Label: I-PER
Word: '##ல்', Predicted Label: I-PER
Word: 'மற்றும்', Predicted Label: O
Word: 'ந', Predicted Label: O
Word: '##ான்', Predicted Label: I-ORG
Word: 'க', Predicted Label: B-LOC
Word: '##ோ', Predicted Label: I-ORG
Word: '##யம்', Predicted Label: I-ORG
Word: '##பு', Predicted Label: I-ORG
Word: '##த்த', Predicted Label: I-ORG
Word: '##ூரி', Predicted Label: I-ORG
Word: '##ல்', Predicted Label: I-ORG
Word: 'வ', Predicted Label: O
Word: '##ரு', Predicted Label: I-ORG
Word: '##கி', Predicted Label: I-ORG
Word: '##ற', Predicted Label: I-ORG
Word: '##ே', Predicted Label: I-ORG
Word: '##ன்', Predict